# Does the linear regression make sense?

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

## RegNetWeb

In [142]:
# import datasets
regnet_regulators = pd.read_csv('regnetworkweb_all8tf.csv')
regnet_targets = pd.read_csv('regnet_targets.csv')
regnet_all = pd.concat([regnet_regulators, regnet_targets], axis=0)

# greater than 100
smaller = pd.read_pickle('test_data.pkl')

# greater than 50
#data = pd.read_pickle('larger_data.pkl')
data = pd.read_pickle('p_values.pkl')



In [143]:
regnet_all

,regulator_symbol,regulator_id,target_symbol,target_id,database,evidence,confidence
0,HIF1A,3091,VHL,7428,"hprd,kegg",Experimental,High
1,HIF1A,3091,TGFB3,7043,"tred,kegg",Experimental,High
2,HIF1A,3091,SLC2A1,6513,"tred,kegg",Experimental,High
3,HIF1A,3091,FIGF,2277,"tred,kegg",Experimental,High
4,HIF1A,3091,VHL,7428,tred,Experimental,High
...,...,...,...,...,...,...,...
27,ETS1,2113,CDK4,1019,kegg,Experimental,High
28,CSDA,8531,CDK4,1019,kegg,Experimental,High
29,RB1,5925,CDK4,1019,"hprd,kegg",Experimental,High
30,RBL1,5933,CDK4,1019,"hprd,kegg",Experimental,High


In [144]:
# capitalize first letter
regnet_all['regulator_symbol'] = [x.capitalize() for x in regnet_all['regulator_symbol']]
regnet_all['target_symbol'] = [x.capitalize() for x in regnet_all['target_symbol']]

In [145]:
data

,predictors,coefficients,target,p_values
42,Mcm3,0.024322,1110004E09Rik,0.037800
61,Mcm6,0.190814,1110004E09Rik,0.035029
81,Cdk4,-0.029327,1500009L16Rik,0.047847
159,Tox,0.059393,1500009L16Rik,0.046285
202,Mcm3,0.077020,1600014C10Rik,0.020605
...,...,...,...,...
88590,Klf2,0.006186,Zranb3,0.015451
88601,Mcm3,0.465177,Zranb3,0.018412
88618,Mcm5,0.149638,Zranb3,0.019094
88660,Hmgb2,-0.088547,Zwilch,0.023983


In [147]:
# to work with data
data = data.reset_index()

In [ ]:
# Curate Data

In [156]:
# Comparing Data from RegNetWeb

predictors = data['predictors'].drop_duplicates().tolist()

summary = {}

for x in predictors:
    t = regnet_all[regnet_all['regulator_symbol']==x]['target_symbol'].tolist()
    d = data[data['predictors']==x]['target'].tolist()
    #print(x)
    #print('This is regnet_all', t)
    #print('This is model', d)
    s = list(set(t) & set(d))
    summary[x]={'total': len(t), 'matchP': s}
    
for x in predictors:
    t = regnet_all[regnet_all['target_symbol']==x]['regulator_symbol'].tolist()
    d = data[data['predictors']==x]['target'].tolist()
    #print(x)
    #print('This is regnet_all', t)
    #print('This is model', d)
    s = list(set(t) & set(d))
    summary[x]['total']+= len(t)
    summary[x]['matchT']= s

In [157]:
summary

{'Mcm3': {'total': 14, 'matchP': ['Cdc45'], 'matchT': []},
 'Mcm6': {'total': 11, 'matchP': [], 'matchT': []},
 'Cdk4': {'total': 11, 'matchP': [], 'matchT': ['Ets1', 'Myc']},
 'Tox': {'total': 0, 'matchP': [], 'matchT': []},
 'Mcm5': {'total': 13, 'matchP': ['Cdc45'], 'matchT': []},
 'Hmgb2': {'total': 1, 'matchP': [], 'matchT': []},
 'Klf2': {'total': 3, 'matchP': [], 'matchT': []},
 'Hif1a': {'total': 118, 'matchP': ['Hk2', 'Nt5e'], 'matchT': []}}

There are a couple matches.. Out of 139 target genes found for ['Cdk4', 'Hif1a', 'Hmgb2', 'Klf2', 'Mcm3', 'Mcm5', 'Mcm6', 'Tox'] there where 4 matches..

The following was the one match looking at the ratio. The p-value of this is pretty high though. 

Looking at about 700 genes with mean/sd values > 50, there is one match, Pdgfb, with a ratio of -61.66.

In [110]:
data[data['target']=='Pdgfb']

,predictors,target,coefficients
183,Hif1a,Pdgfb,-61.660615
594,Mcm6,Pdgfb,-70.004292


check coefficients without dividing by the variation -> look at absolutely high variation coefficients
final list of avarage coefficients: divide all values by average coefficient -> higher and lower rank..
treat coefficients as score to calculate z-scores and get pvalues -> threshold can be significance. 

# GSEA

In [113]:
smaller = data.reset_index()

In [121]:
smaller['target'] = [x.upper() for x in smaller['target']]

In [122]:
smaller[['target', 'coefficients']].to_csv('greater50.csv')

# Over representation Analysis ORA